Ferramentas de NLP usando o dataset de tweets

Modelo KNN

Pré Processamento

80/20 Treinamento


In [44]:
from google.colab import drive
from collections import Counter
import pandas as pd
import nltk
import re
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize.casual import casual_tokenize
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import numpy as np

Dando permissão para o colab poder acessar os arquivos do meu drive

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
df_train = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/test.csv')


# df.head()

X_df_test = df_test.Text.values
y_df_test = np.zeros([1640,])

X_df_train = df_train.Text.values
y_df_train = df_train.Classificacao.values


X_df_train = array(X_df_train)
X_df_test = array((X_df_test))
y_df_train = array(y_df_train)
y_df_test = array(y_df_test)

X = np.append(X_df_train,X_df_test)
y = np.append(y_df_train,y_df_test)

# y = pd.get_dummies(y)

Para Testar a acuracia no meu conjunto de treino.

In [54]:
X = df_train.Text
y = df_train.Classificacao

In [ ]:
X.shape

(8199,)

In [70]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

bags_of_words = []
vocabulary = []


# tokens = [word for word in tokens if word.isalnum()] # Retira tudo de pontuação e link

for text in X:
  sentence = text
  # sentence = re.sub(r'http\S+', '', text)
  tokens = casual_tokenize(sentence)
  # puncs = set((',', '.', '--','-','!','?',':', ';', '``', "''", '(', ')', '[', ']','\u2026','"'))
  # tokens = (x.lower() for x in tokens if x not in puncs)
  tokens = [x2 for x2 in tokens if x2 not in stop_words]
  bags_of_words.append(Counter(tokens))


df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)

df_bows.shape


(6559, 12405)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(df_bows,y, test_size = 0.2, random_state = 109,shuffle=False)

In [78]:
clf = svm.SVC(decision_function_shape="ovr", kernel= "linear")
# clf = LogisticRegression()
# svc = KNeighborsClassifier(n_neighbors= 1, metric='euclidean')

# clf = tree.DecisionTreeClassifier()
# clf = SGDClassifier(loss='hinge')
# clf =SGDClassifier(loss='hinge',max_iter=10000,penalty='elasticnet',shuffle=False)




clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [79]:
predict = clf.predict(X_test)

In [80]:
from sklearn.metrics import accuracy_score as acs

print(acs(predict, y_test))

0.958079268292683


In [51]:
clf1 = svm.SVC(decision_function_shape="ovr", kernel= "linear")
clf2 = tree.DecisionTreeClassifier()
clf3 = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
clf4 = LogisticRegression()

estimators = []
estimators.append(('SVM', clf1))
estimators.append(('Tree', clf2))
estimators.append(('KNN', clf3))
estimators.append(('Regressão Logistica', clf4))
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train,y_train)

VotingClassifier(estimators=[('SVM',
                              SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='linear', max_iter=-1,
                                  probability=False, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False)),
                             ('Tree',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_de...
                                                   n_jobs=None, n_neighbors=1,
                                                   p=2, weights='uniform')),
                             ('Regressão Logistica',

In [ ]:
final = pd.DataFrame(df_test.Id)
final['Category'] = predict

final.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/finalSGD22.csv',index=False)

# df.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/final.csv', index=False)

In [ ]:
predict.shape

(1640,)

In [ ]:
df_test.Id

0       3568
1       1323
2       7976
3       2408
4       4435
        ... 
1635    3536
1636    6881
1637     627
1638    2165
1639    4268
Name: Id, Length: 1640, dtype: int64

In [ ]:
df

,Category,Id
0,Negativo,3568
1,Neutro,1323
2,Positivo,7976
3,Negativo,2408
4,Negativo,4435
...,...,...
112,Positivo,5279
113,Positivo,6786
114,Negativo,3835
115,Negativo,2859
